# Recursion Schemes
Based on ***Fantastic Morphisms and Where to Find Them***.

Recursion is a way of defining a function on any mathematical object which is “defined inductively”. Yet,
left on its own, an inductive definition might lead to problems, such as infinite looping (non termination).
Thereforem, a recursion scheme is a structure recursion that by its construction guarantees termination.
For example, where one writes a for-loop in Julia, there is no guarantee that the loop is ever going to end.
Yet, in functional programming, a `foldl` is guaranteed to finish.
Hence, recursion schemes are a collection of implementations that try to formalize the different recursion operations.

There are several examples of such recursions, each one with its name and structure. The paper
**Fantastic Morphisms and Where to Find Them** has a collection of such schemes and a table
describing their usage. For example, catamorphisms are usade to consume inductive data. Anamorphisms
are used to generate coinductive data. Accumulation is used in recursions with an accumulative parameter.
And so on.

Each of these recursion schemes have a different category theoretic formalism attached to them.
For example, a catamorphism is the recursion scheme derived using initial $F$-algebras. The anamorphism
is obtained from terminal co-algebras, and so on.

In [1]:
using Pkg
Pkg.activate(".")
using MLStyle

  Activating project at `~/Documents/Julia_Tutorials/CategoryTheory`


## 1. Catamorphism

Let us start with an implementation of catamorphisms.
First, consider a functor `data NatF x = ZeroF | SuccF x`. 

An example of a NatF-algebra is `(fib,(Int,Int))` where `fib : NatF(Int,Int)-> (Int,Int)`.

In [2]:
@data NatF{a} begin
    ZeroF()
    SuccF(s::a)
end
ZeroF() = ZeroF{Union{}}()

fmap(f::Function, a::ZeroF) = a
fmap(f::Function, a::SuccF) = SuccF(f(a.s))

fib(x::ZeroF) = (1,1)
fib(x::SuccF{Tuple{Int,Int}}) = (x.s[2],x.s[1] + x.s[2])

z = ZeroF{Tuple{Int,Int}}()
@show fib(z)
@show fib(SuccF((1,2)));

fib(z) = (1, 1)
fib(SuccF((1, 2))) = (2, 3)


Now, consider the category of `NatF` algebras. The initial object for this category
is an algebra `(fix, Nat)` such that `fix: NatF Nat -> Nat`.
Note that Lambek's theorem states that there exists `unfix:Nat -> NatF Nat` which is the inverse
function of `fix`.

The idea here is that `NatF Nat` and `Nat` are isomorphic. Hence,
`fix` takes `NatF Nat` to `Nat`, and `unfix` does the opposite.

In [21]:
# struct Fix{F}
#     _1::F
# end
# fmap(f::Function, x::Fix) = Fix(fmap(f, x._1))

# Fix(ZeroF())
# Fix(SuccF(10))

# unfix(x::Fix{<:ZeroF}) = ZeroF{Fix{NatF}}()
# unfix(x::Fix{<:SuccF}) = SuccF(x._1.s)

# fix(n::ZeroF{<:Fix}) = Fix(ZeroF())
# fix(n::SuccF{<:Fix}) = Fix(SuccF(n.s))


# unfix(Fix(ZeroF()))
# x = Fix(SuccF(Fix(ZeroF())))
# @show fix(unfix(x)) == x

# z = Fix(ZeroF())
# @show fix(unfix(z)) == z;

# cata(alg::Function, x::Fix) = alg(fmap(y -> cata(alg, y), unfix(x)))

# x = Fix(SuccF(
#         Fix(SuccF(
#                 Fix(SuccF(
#                         Fix(ZeroF())
#                     ))
#                 ))
#         ))
# cata(fib,x)

In [22]:
@data Nat begin
    Zero()
    Succ(s::Nat)
end
fmap(f::Function, a::Zero) = a
fmap(f::Function, a::Succ) = Succ(f(a.s))

unfix(n::Zero) = ZeroF{Nat}()
unfix(n::Succ) = SuccF(n.s)
fix(n::ZeroF{<:Nat}) = Zero()
fix(n::SuccF{<:Nat}) = Succ(n.s)

@show unfix(Succ(Zero())) isa NatF{<:Nat}
@show unfix(Zero()) isa NatF{<:Nat};

@show unfix(fix(ZeroF{Nat}())) == ZeroF{Nat}()
@show fix(unfix(Zero())) == Zero();

v = Succ(Succ(Succ(Zero())))
fix(unfix(v)) == v

unfix(Succ(Zero())) isa NatF{<:Nat} = true
unfix(Zero()) isa NatF{<:Nat} = true
unfix(fix(ZeroF{Nat}())) == ZeroF{Nat}() = true
fix(unfix(Zero())) == Zero() = true


true

In [23]:
cata(alg::Function, x) = alg(fmap(y -> cata(alg, y), unfix(x)))

x = Succ((
        Succ(
            Succ(Zero())
            ))
    )
cata(fib,x)

(3, 5)

## 2. Anamorphism

In [24]:
getfirst(l::Vector) = length(l) > 0 ? l[begin] : l
getrest(l::Vector{T}, n=2) where T= length(l) > 1 ? l[2:end] : T[]

@data TreeF{x} begin
    LeafF()
    NodeF(::Int, ::x, ::x)
end
LeafF() = LeafF{Union{}}()
fmap(f::Function, x::LeafF) = x
fmap(f::Function, x::NodeF) = NodeF(x._1, f(x._2), f(x._3))

@data Tree begin
    Leaf()
    Node(::Int, ::Tree, ::Tree)
end
# LeafF() = LeafF{Union{}}()
fmap(f::Function, x::Leaf) = x
fmap(f::Function, x::Node) = Node(x._1, f(x._2), f(x._3))

unfix(x::Leaf) = LeafF{Tree}()
unfix(x::Node) = NodeF(x._1,x._2,x._3)
fix(x::LeafF{<:Tree}) = Leaf()
fix(x::NodeF{<:Tree}) = Node(x._1,x._2,x._3)
NodeF(i::Int,x::Tree,y::Tree) = NodeF{Tree}(i,x,y)

splitting(x::Vector) = begin 
    if length(x) == 0
       return LeafF()
    end
    n = getfirst(x)
    ns= getrest(x)
    NodeF(n, filter(x-> x ≤ n, ns), filter(x->x>n, ns))
end

splitting([])
splitting([4,2,3,5,6,0,1])


ana(coa::Function, x) = fix(fmap(y -> ana(coa, y), coa(x)))

ana (generic function with 1 method)

In [25]:
l = [4,2,4,1,2,4,6,7]

ana(splitting, l)

Node(4, Node(2, Node(1, Leaf(), Node(2, Leaf(), Leaf())), Node(4, Node(4, Leaf(), Leaf()), Leaf())), Node(6, Leaf(), Node(7, Leaf(), Leaf())))

In [26]:
tolist(x::LeafF) = Int[]
tolist(x::NodeF) = vcat(x._2,[x._1],x._3)

qsort(x::Vector{Int}) = cata(tolist, ana(splitting,x))

qsort([2,5,3,1,0,4])
qsort([1,2])

2-element Vector{Int64}:
 1
 2

In [27]:
# unfix(x::Fix{<:LeafF}) = LeafF{Fix{TreeF}}()
# unfix(x::Fix{<:NodeF}) = NodeF{Fix{TreeF}}(x._1,x._2,x._3)

# fix_(n::LeafF{<:Fix{<:TreeF}}) = Fix(LeafF())
# fix_(n::NodeF{<:Fix{<:TreeF}}) = Fix(NodeF(n._1,n._2,n._3))
# fix_(n::LeafF{Fix}) = Fix(LeafF())
# fix_(n::NodeF{Fix}) = Fix(NodeF(n._1,n._2,n._3))

# NodeF(i::Int,x::Fix,y::Fix) = NodeF{Fix}(i,x,y)
# ana_(coa::Function, x) = fix_(fmap(y -> ana_(coa, y), coa(x)))
# cata(tolist,ana_(splitting, [1,2]))